In [ ]:
# Import dependencies
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime as dt, timedelta

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
# Connection Variable 
connection = engine.connect()

In [ ]:
# Checking out the measure table
measure_df = pd.read_sql("SELECT * FROM measurement", connection)
measure_df

In [ ]:
# Checking out the station table
station_df = pd.read_sql("SELECT * FROM station", connection)
station_df

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
recent_date = session.query(measurement.date).order_by(measurement.date.desc()).first()
recent_date

In [ ]:
# Converting date as string
year_ago = dt.strptime(recent_date[0], "%Y-%m-%d") # date to string
year_ago = year_ago - timedelta(days = 365) # subtracting date minus days in a year
year_ago = year_ago.strftime("%Y-%m-%d") # string to date
year_ago

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
year_ago = dt.strptime(recent_date[0], "%Y-%m-%d") # date to string
year_ago = year_ago - timedelta(days = 365) # subtracting date minus days in a year
year_ago = year_ago.strftime("%Y-%m-%d") # string to date
# year_ago
# Perform a query to retrieve the date and precipitation scores
precip_year_ago = session.query(measurement.date, measurement.prcp).filter(measurement.date >= year_ago).all()
# precip_year_ago
# Save the query results as a Pandas DataFrame and set the index to the date column
recent_rain = pd.DataFrame(precip_year_ago)
recent_rain.set_index("date", inplace = True)
#recent_rain.head()

# Sort the dataframe by date
recent_rain.sort_index(inplace = True)
#recent_rain.head()

# Use Pandas Plotting with Matplotlib to plot the data
recent_rain.plot(figsize = (10,5), legend = False)
plt.title("Last 12 Months of Precipitation Data")
plt.ylabel("Precipitation Amount")
plt.xlabel("Date") 


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
recent_rain.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
station_numbers = session.query(station.station).count()
station_numbers

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
station_counts = session.query(measurement.station, func.count(measurement.station)).\
group_by(measurement.station).order_by(func.count(measurement.station).desc())
# name the objects you'll be calling
most_active_station = station_counts[0][0]
active_amount = station_counts[0][1]

# active_amount
# most_active_station

In [ ]:
# Finding most active stations
print(f"The most active station is {most_active_station}, with {active_amount} data points.")

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
active_data = session.query(func.min(measurement.tobs), func.max(measurement.tobs), func.avg(measurement.tobs)).\
filter(measurement.station == station_counts[0][0])
# active_data[0]
minimum = active_data[0][0]
# minimum
maximum = active_data[0][1]
# maximum
average = active_data[0][2]
# average


In [ ]:
print(f"The lowest temperature from station {most_active_station} is {minimum}, the highest temperature is {maximum}, and the average temperature is {average}.")

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
active_data = session.query(measurement.tobs, measurement.date).\
filter(measurement.station == station_counts[0][0]).filter(measurement.date >= year_ago)
active_df = pd.DataFrame(active_data)
active_df.plot(kind = "hist", bins = 12, legend = False, figsize = (10, 10))

plt.title("Last 12 Months of Temperature Observation Data from the Most Active Station")
plt.xlabel("Temperature")
plt.ylabel("Frequency of Temperature")

# Close session

In [ ]:
# Close Session
session.close()